# CP322 Assignment 3
### Riley Huston (190954880) | Samson Goodenough (190723380)

## 1 Concept Qustions
### 1.1 Neural Nets
##### 1.1.1 What is the output $P (y = 1|x, w)$ from the above neural net? Express it in terms of $x_i$, $c$ and weights $w_i$

$$g(z) = \frac{1}{1+e^{-z}}~~~and~~~h(z)=cz$$
$$
P(y=1|x,w) = g(w_7 + w_8h(w_1+w_3x_1+w_5x_2)+w_9h(w_2+w_4x_1+w_6x_2)) \\
= \frac{1}{1+e^{-(w_7 + w_8h(w_1+w_3x_1+w_5x_2)+w_9h(w_2+w_4x_1+w_6x_2))}} \\
= \frac{1}{1+e^{-(w_7 + w_8(cw_1 + cw_3x_1 + cw_5x_2)+w_9(cw_2 + cw_4x_1 + cw_6x_2))}} \\
= \frac{1}{1+e^{-(w_7 + cw_1w_8 + cw_3w_8x_1 + cw_5w_8x_2 + cw_2w_9 + cw_4w_9x_1 + cw_6w_9x_2)}} \\
= \frac{1}{1+e^{-(w_7 + cw_1w_8 + cw_2w_9 + x_1(cw_3w_8 + cw_4w_9) + x_2(cw_5w_8 + cw_6w_9))}}
$$

##### 1.1.2 Is it true that any multi-layered neural net with linear activation functions at hidden layers can be represented as a neural net without any hidden layer? Briefly explain your answer
$$
\boxed{
  \begin{aligned}
    \text{Yes it is true since any network that uses a linear activation function in any hidden layers can simply be chalked up to additional inputs to the final output layer.}\\
    \text{This is the same for the example above, now that we have the output it can easily be rewritten to have no hidden layers as seen below.}
  \end{aligned}
}
$$
![](./assets/diagram.png)
### 1.2 Support Vector Machines
##### 1.2.1 How large can $h ≥ 0$ be so the training examples are still linearly separable
$$\boxed{h \le 1}$$
##### 1.2.2 Will the direction of the maximum margin decision boundary change as a function of h when the samples are separable? Explain your answer with one sentence 
$$\boxed{\text{No, because }x_1, x_2, x_3 \text{ remain as the support vectors}}$$

##### 1.2.3 What will be the margin obtained by the maximum margin boundary as a function of h? Note that the margin as a function of $h$ is actually a linear function 
$$
\boxed{ 
  \frac{1-h}{\sqrt{2}}~|~0 \le h \le 1, \text{otherwise }0
}
$$

## 2 Programming Questions


In [1]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.losses import SparseCategoricalCrossentropy 
import plotly.graph_objects as go

### 2.1 Image Pre-Processing

In [2]:
train_datagen=ImageDataGenerator(
    zoom_range=0.2, # Zooming rate of the image
    horizontal_flip=True, # Make a horizontal copy
    rescale=1.0/255.0, # Normalize the new images
    width_shift_range=0.10, # Percentage of width shifting
    height_shift_range=0.10, # Percentage of height shifting
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory("xray_dataset_covid19/train", target_size=(64,64), batch_size=32, class_mode='binary')

validation_generator = validation_datagen.flow_from_directory("xray_dataset_covid19/test", target_size=(64,64), batch_size=32, class_mode='binary')

Found 148 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


### 2.2 Model Building


In [3]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10))

model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


### 2.3 Model Evaluation

In [4]:
EarlyStop = EarlyStopping(patience=6)
history = model.fit(train_generator, epochs=20, validation_data = validation_generator, callbacks=[EarlyStop])


Epoch 1/20
5/5 [==============================] - 3s 648ms/step - loss: 1.5217 - accuracy: 0.3986 - val_loss: 0.7141 - val_accuracy: 0.5000
Epoch 2/20
5/5 [==============================] - 2s 433ms/step - loss: 0.6262 - accuracy: 0.6824 - val_loss: 0.3752 - val_accuracy: 0.9000
Epoch 3/20
5/5 [==============================] - 2s 475ms/step - loss: 0.4112 - accuracy: 0.8243 - val_loss: 0.1791 - val_accuracy: 0.9750
Epoch 4/20
5/5 [==============================] - 2s 464ms/step - loss: 0.4100 - accuracy: 0.8176 - val_loss: 0.4852 - val_accuracy: 0.7250
Epoch 5/20
5/5 [==============================] - 2s 478ms/step - loss: 0.4481 - accuracy: 0.8243 - val_loss: 0.1698 - val_accuracy: 0.9250
Epoch 6/20
5/5 [==============================] - 2s 447ms/step - loss: 0.4446 - accuracy: 0.7905 - val_loss: 0.3038 - val_accuracy: 0.8750
Epoch 7/20
5/5 [==============================] - 2s 520ms/step - loss: 0.3901 - accuracy: 0.8176 - val_loss: 0.1306 - val_accuracy: 0.9500
Epoch 8/20
5/5 [====

In [5]:
fig = go.Figure(data=go.Line(x=history.epoch, y=history.history['accuracy'], name='Accuracy'), layout=go.Layout(title='Accuracy', xaxis=dict(title='Epoch'), yaxis=dict(title='Accuracy')))
fig.add_trace(go.Line(y=history.history['val_accuracy'], name="Validation_Accuracy"))
fig.show()

C:\Users\mrrjh\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning: plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.

  warnings.warn(
C:\Users\mrrjh\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [6]:
fig = go.Figure(data=go.Line(x=history.epoch, y=history.history['loss'], name='Loss'), layout=go.Layout(title='Loss', xaxis=dict(title='Epoch'), yaxis=dict(title='Loss')))
fig.add_trace(go.Line(y=history.history['val_loss'], name="Validation_Loss"))
fig.show()